In [148]:
import numpy as np 
from scipy.optimize import minimize
from scipy.fft import fft, ifft
from scipy.stats import multivariate_normal
from scipy.linalg import circulant, block_diag
from numpy.random import default_rng

I generated the joint Gaussian data by using BCCB (block circulant with circulant blocks) covariance matrix as known covariance and mean (mu_k = a + b*k). It allows for easier analysis after applying the FFT to the data due to the specific structure of the data and its covariance.  

This idea from this paper: : Jonathan R. Stroud, Michael L. Stein & Shaun Lysen (2017) Bayesian and
Maximum Likelihood Estimation for Gaussian Processes on an Incomplete Lattice, Journal of
Computational and Graphical Statistics, 26:1, 108-120, DOI: 10.1080/10618600.2016.1152970

In [149]:
# Data generation
n = 500
a_true = 10
b_true = 6
k = np.linspace(0, n-1, n)
mean_true = a_true + b_true * k
rng = default_rng(12345)
variance = 1.0 

# Create block circulant covariance matrix
block_size = 2  # For 2x2 circulant blocks
if n % block_size != 0:
    raise ValueError("n must be divisible by block_size")
num_blocks = n // block_size
base_block = circulant([variance, -variance])  # Create a 2x2 circulant block
blocks = [base_block for _ in range(num_blocks)]
block_toeplitz = block_diag(*blocks)

# Make the covariance matrix positive definite
eigenvalues = np.linalg.eigvals(block_toeplitz)
max_eigenvalue = np.max(np.abs(eigenvalues))
cov = block_toeplitz + (max_eigenvalue * 1.1) * np.eye(n) 
y = rng.multivariate_normal(mean_true, cov, size=None)


Problem 1: Given $y_1,y_2,\ldots,y_N$ are joint Gaussian with mean $\mathbb{E}[y_k] = \mu_k$ and the variance is known $\Sigma$. if $y_k = a + b*k$, how to infer $a$ and $b$

In this problem, if we want to infer the parameters, we can consider Maximum Likelihood Estimation which can effectively maximize the probability of a given data set. 

Firstly, Let $\theta = [a,b]$. the mean is $\mu_k = a + b*k$ and $\Sigma$ is known. 

I can choose the negetive log-likelihood $-\ln(L(\theta|y))  = -\frac{N}{2}\ln(2\pi) + \frac{1}{2}\ln|\Sigma| + \frac{1}{2} (y - \mu(\theta))^T \Sigma^{-1} (y - \mu(\theta))$ from the following reference

In [150]:
def neg_log_likelihood(theta):
    a, b = theta
    mu = a + b*k
    diff = y - mu
    return -n/2*np.log(2*np.pi) + 0.5*np.log(np.linalg.det(cov)) + 0.5*diff.T @ np.linalg.inv(cov) @ diff

In [151]:
# Initial guess for a and b
theta0 = [0, 0]

# Optimize
res = minimize(neg_log_likelihood, theta0)
print("Estimated a and b:", res.x)

Estimated a and b: [9.72247909 6.00093156]


From here, we can see the estimated a and b are almost same as the true a and b that we set. Also, we can get more accurate result by incresing the data size.

Reference: https://www.hashpi.com/maximum-likelihood-estimation-and-loss-function-design

Problem 2: Given the Fourier transform of $y_1,y_2,\ldots,y_N$ are $z_1,z_2,\ldots,z_N$ , how to infer $a$ and $b$

In [152]:
# FFT of the data 
y_fft = fft(y,axis=0)

The first approach: we can just use the Inverse Fast Fourier Transfrom for the data set. We can see that results are same.

In [153]:
def neg_log_likelihood_complex(theta):
    a, b = theta
    mu = a + b * k
    diff = ifft(y_fft).real - mu  
    return -n/2*np.log(2*np.pi) + 0.5*np.log(np.linalg.det(cov)) + 0.5*np.dot(diff.T, np.linalg.solve(cov, diff))

In [154]:
# Initial guess for a and b
theta0 = [0, 0]

# Optimize
res = minimize(neg_log_likelihood_complex, theta0)
print("Estimated a and b from FFT data:", res.x)

Estimated a and b from FFT data: [9.72247943 6.00093155]


In this approach, We did not access anything from the FFT data. I will explore other approch below trying to firgue out a method in FFT data. 

The second approach

I applied the EM algorithm for analysis the FFT of the given data. Since the goal is to find the parameters $\theta = [a,b]$ that maximize the expected log-likelihood. This is done iteratively using two steps:

E-step: Compute the expected log-likelihood, given the current parameter estimate.
$Q(θ|θ^t) = E[log p(Z, Y|θ) | Y, θ^t]$

M-step: Update the parameter estimate by maximizing the expected log-likelihood.
$θ^t = argmax_{θ} Q(θ|θ^t)$

In the code, I turned a maximization problem into a minimization problem.

In [155]:
# Perform FFT on data and covariance
fft_C = fft(cov)
fft_C_diag = np.diag(fft_C)

# Define log-likelihood function for Monte Carlo EM
def log_likelihood_MC(theta, Z):
    a, b = theta
    mu = a + b * k
    mu_fft = fft(mu)
    diff_fft = Z - mu_fft
    return -0.5 * np.mean(np.abs(diff_fft)**2 / np.abs(fft_C_diag)) - 0.5 * np.sum(np.log(np.abs(fft_C_diag)))

# E-step:generating Monte Carlo samples from the FFT-transformed data
num = 500
Z_MC = np.array([y_fft + fft(rng.multivariate_normal(np.zeros(n), fft_C.real, size=None)) for _ in range(num)])

# Initial guess for a and b
theta0 = [0, 0]

# Optimize (M-step)
res = minimize(lambda theta: -log_likelihood_MC(theta, Z_MC), theta0)

print("Estimated a and b from FFT data:", res.x)


/var/folders/lj/02lhd2x95j5cdgjwf20qlnc40000gn/T/ipykernel_72366/2884421372.py:15: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  Z_MC = np.array([y_fft + fft(rng.multivariate_normal(np.zeros(n), fft_C.real, size=None)) for _ in range(num)])


Estimated a and b from FFT data: [9.7148829  6.00093721]


From this method, I obtained the a and b are same as previous. 